# Problème n°2: PointNet

Certains jeux de données impliquent des nuages de points dans un espace 3D. Penser par exemple à un ensemble de mesures lidar : chaque tir permet de renseigner les coordonnées d'un des points de l'objet ciblé.
Une tâche intéressante consiste à classer chacun des points du nuage en fonction de l'objet auquel il appartient. Cette tâche est considérée comme une variante de la segmentation sémantique d'images.

Ce problème introduit à une méthode directe de segmentation d'un nuage par deep learning. Elle est basée sur une architecture particulière appelée PointNet. \
Dans la première partie, on présente un jeu de données (synthétisé à la volée) impliquant des nuages de points.
Dans la seconde partie, on explore la structure et les propriétés de PointNet. Dans la troisième, on l'entraîne et dans la dernière partie, on charge les poids d'une version améliorée de PointNet (PointNet++) pour comparaison.

La cellule suivante permet les imports nécessaires:

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import matplotlib.pyplot as plt
import os
! pip install einops
! git clone https://github.com/badreouh/exam_2025.git
! cp exam_2025/utils/utils_probleme2.py .
from utils_probleme2 import gen_pointcloud, plot_triplets

## Partie I : un exemple de PointCLoud data

Pour construire le jeu de données, on simule un terrain couvert de deux types de bâtiments : des immeubles de forme rectangulaire aux toits plats et des igloos (dômes). Pour créer les nuages, on échantillonne les surfaces vues du ciel (les murs des bâtiments rectangulaires ne sont pas échantillonnées), en favorisant les zones d'altitude non nulles.
Le but est de distinguer les igloos du reste (sol et toits des bâtiments). Il s'agit donc d'une segmentation binaire.

In [ ]:
batch_size = 6
input_points, target_list, target_points  = gen_pointcloud(batch_size)


for i in range(batch_size):
  print(i)
  # Représentation 3D des nuages de points et
  # les paramètres elev et azim permettent de changer l'angle de vue
  plot_triplets(input_points[i].transpose(0,1).cpu(),
                elev=75, azim=0)

  # Cibles : les points appartenant aux toitures d'igloos sont
  # dans la classe 1, les autres, dans la classe 0.
  plot_triplets(target_points[i].transpose(0,1).cpu(),
                title='Cibles',
                cbar_label='classe')

  # Note: target_points contient non seulement les classes
  # mais aussi les coordonnées x et y des points, pour
  # faciliter leur visualisation

**Q1** A quoi correspondent les différentes dimensions de *input_points* ?

Réponse :

Les différentes dimensions de input_points correspondent aux éléments suivants :

Dimension 0 : batch_size (taille du lot)

Description : Cette dimension représente le nombre d'exemples (nuages de points) traités simultanément dans une itération. Dans votre cas, batch_size = 6, ce qui signifie que 6 nuages de points sont traités en parallèle.
Dimension 1 : caractéristiques (features)

Description : Cette dimension correspond aux différentes caractéristiques ou attributs de chaque point dans le nuage. Généralement, pour des nuages de points en 3D, cette dimension est de taille 3, représentant les coordonnées spatiales :
0 : Coordonnée X
1 : Coordonnée Y
2 : Coordonnée Z
Cependant, si des attributs supplémentaires sont inclus (comme des couleurs, des intensités, etc.), cette dimension peut être étendue en conséquence.
Dimension 2 : nombre de points

Description : Cette dimension indique le nombre total de points dans chaque nuage de points. Par exemple, si chaque nuage contient 1024 points, cette dimension sera de taille 1024.
Résumé des dimensions de input_points :

input_points a une forme généralement de (batch_size, num_features, num_points).
batch_size : Nombre de nuages de points dans le lot (ex. 6).
num_features : Nombre de caractéristiques par point (ex. 3 pour X, Y, Z).
num_points : Nombre total de points dans chaque nuage de points.
Illustration avec un exemple :

Supposons que input_points a la forme (6, 3, 1024) :

6 : Il y a 6 nuages de points dans le lot.
3 : Chaque point a 3 caractéristiques (X, Y, Z).
1024 : Chaque nuage contient 1024 points.
Cette structure permet au modèle de traiter efficacement les données de nuages de points en exploitant à la fois les informations spatiales et la parallélisation offerte par le traitement par lots.

**Q2** Les points d'un nuage sont-ils rangés dans un ordre particulier ?

Réponse :

Non, les points d’un nuage de points ne sont généralement pas rangés dans un ordre particulier. Un nuage de points est considéré comme un ensemble non ordonné de points dans l’espace. Cela signifie que l’ordre dans lequel les points sont listés ou stockés n’a pas d’importance intrinsèque pour la représentation géométrique ou pour les algorithmes qui traitent ces données.

Explications supplémentaires :

Invariance à l’ordre :

Les réseaux de neurones conçus pour traiter les nuages de points, tels que PointNet ou PointNet++, sont spécialement architecturés pour être invariants à l’ordre des points. Cela signifie que la permutation des points d’entrée ne doit pas affecter la sortie du modèle. Cette propriété est essentielle car, dans la réalité, les points capturés par des capteurs (comme les LIDAR) n’ont pas d’ordre spécifique.
Traitement des données :

Lors de la préparation des données pour l’entraînement, les points peuvent être réorganisés aléatoirement ou selon certaines stratégies (comme le sous-échantillonnage) pour augmenter la robustesse du modèle. Cependant, cette réorganisation ne confère aucune signification particulière à l’ordre des points.
Visualisation :

Lors de la visualisation des nuages de points, l’ordre des points peut influencer le rendu graphique (par exemple, l’ordre de superposition des points), mais cela n’a aucune incidence sur les propriétés géométriques ou les caractéristiques apprises par le modèle.
Stockage et efficacité :

Du point de vue du stockage et de l’efficacité computationnelle, traiter les nuages de points comme des ensembles non ordonnés permet de simplifier les opérations et d’optimiser les performances des algorithmes d’apprentissage automatique.

**Q3** (question ouverte). Si vous deviez traiter le problème avec un FCN ou un ViT (Visual Transformer), que feriez-vous ?

1. Utilisation d'un FCN (Fully Convolutional Network)
Les FCN sont traditionnellement conçus pour traiter des données structurées en grille, comme les images. Pour les appliquer aux nuages de points 3D, plusieurs étapes d'adaptation sont nécessaires :

a. Conversion des Nuages de Points en Grille Voxelisée :
Voxelisation : Transformer le nuage de points en une représentation voxelisée 3D où l'espace est divisé en petits cubes (voxels). Chaque voxel peut contenir des informations telles que la présence ou l'absence de points, ou des caractéristiques supplémentaires comme la densité.
Avantages : Permet d'utiliser directement des architectures de CNN 3D existantes.
Inconvénients : La voxelisation peut entraîner une perte de précision et une augmentation significative de la mémoire requise, surtout pour des résolutions élevées.
b. Projection Multi-Vues :
Approche Multi-Vues : Projeter le nuage de points sous différents angles (par exemple, vues frontale, latérale et supérieure) pour créer des images 2D.
Traitement avec FCN 2D : Appliquer un FCN standard sur chaque vue 2D pour effectuer la segmentation, puis fusionner les résultats des différentes vues.
Avantages : Réduit la complexité par rapport à la voxelisation 3D et permet d'exploiter des architectures CNN bien établies.
Inconvénients : Peut introduire des artefacts de projection et ne capture pas pleinement la structure 3D originale.
c. Utilisation de Convolutions Spécifiques aux Points :
Adaptation des Convolutions : Développer ou utiliser des couches de convolution adaptées aux données non structurées des nuages de points, inspirées par des architectures comme PointCNN ou SparseConvNet.
Intégration dans un FCN : Incorporer ces couches spécialisées dans une architecture FCN pour maintenir la capacité de segmentation tout en traitant efficacement les données 3D.
Avantages : Meilleure préservation des informations géométriques spécifiques aux nuages de points.
Inconvénients : Complexité accrue et nécessité de concevoir des couches personnalisées.
2. Utilisation d'un ViT (Visual Transformer)
Les Transformers, et plus particulièrement les ViT, ont montré des performances remarquables dans divers domaines de la vision par ordinateur. Cependant, leur application directe aux nuages de points 3D nécessite des adaptations spécifiques :

a. Projection en Images ou Utilisation de Vues Multi-Scales :
Projection Multi-Vues : Comme pour les FCN, projeter le nuage de points en plusieurs vues 2D et traiter chaque vue avec un ViT.
Intégration des Informations Multi-Vues : Utiliser des mécanismes d'attention pour fusionner les informations provenant de différentes projections, améliorant ainsi la compréhension globale du nuage de points.
Avantages : Tirer parti des capacités d'attention globale des Transformers pour capturer des relations complexes entre différentes parties du nuage.
Inconvénients : Les mêmes limitations que pour les FCN en termes de perte de précision et de complexité de fusion des vues.
b. Traitement Direct des Points avec des Transformers Adaptés :
Point Transformer : Utiliser des architectures de Transformers spécifiquement conçues pour les nuages de points, telles que le Point Transformer, qui intègrent des mécanismes d'attention localisés et globalisés adaptés aux données non ordonnées.
Encodage des Caractéristiques : Incorporer des encodages positionnels ou des descripteurs géométriques pour permettre au Transformer de capturer les relations spatiales entre les points.
Avantages : Exploitation directe des capacités des Transformers à modéliser des dépendances complexes sans nécessiter de conversion en grilles.
Inconvénients : Complexité computationnelle élevée et besoin de grandes quantités de données pour un entraînement efficace.
c. Intégration avec des Techniques Hybrides :
Combinaison avec des Réseaux de Neurones Convolutifs : Utiliser des CNN pour extraire des caractéristiques locales avant de les passer à un Transformer pour modéliser les relations globales.
Avantages : Combiner les forces des CNN (extraction de caractéristiques locales efficaces) avec celles des Transformers (modélisation des dépendances globales).
Inconvénients : Augmentation de la complexité du modèle et des exigences en matière de calcul.
Considérations Générales :
Invariance à l'Ordre des Points : Tant pour les FCN que pour les ViT, il est crucial de s'assurer que la représentation utilisée respecte l'invariance à l'ordre inhérente aux nuages de points.

Efficacité Computationnelle : Les Transformers, en particulier, peuvent être gourmands en ressources. Des techniques telles que la réduction de la dimensionnalité ou l'utilisation de mécanismes d'attention plus efficaces peuvent être nécessaires.

Qualité des Représentations : La manière dont les nuages de points sont représentés (voxelisation, projections, encodages spéciaux) a un impact significatif sur les performances des modèles.

## Partie II : le modèle PointNet

Dans cette partie, on s'intéresse à la propriété principale d'un réseau PointNet : l'utilisation d'opérations invariantes par rapport à l'ordre dans lequel les points sont présentés au réseau.

In [ ]:
from utils_probleme2 import PointNetSegHead
pointnet = PointNetSegHead(num_points=800, num_global_feats=1024, m=2).cuda()

input_points, target_list, _ = gen_pointcloud(batch_size)
input_points = input_points.cuda()
output, _, _ = pointnet(input_points)

**Q1** La sortie du modèle PointNet correspond au premier tenseur du *tuple* fourni la fonction *forward* de *pointnet*. A quoi correspondent les différentes dimensions de *output* ? Quel est l'effet d'une permutation des points contenus dans *inputs_points* sur la sortie ? Répondre :

- en vous référant à l'article [l'article](https://arxiv.org/abs/1612.00593) qui introduit ce réseau (citer dans le texte).
- à partir de tests à effectuer dans la cellule de code suivante (utiliser torch.randperm pour générer des permutations sur les entrées)

**Q2** L'architecture de *pointnet* est décrite dans la Figure 2 de l'article (voir ci-dessous) évoqué à la question précédente. En dehors des opérations notées "input transform" et "feature transform", dont la compréhension est plus délicate, quelles sont les différentes opérations conduisant à une segmentation ? Que signifie le terme "shared" et expliquer en quoi ces opérations sont invariantes par rapport à l'ordre de présentation des points.

<img src= https://miro.medium.com/v2/resize:fit:1100/format:webp/1*lFnrIqmV_47iRvQcY3dB7Q.png >

## Partie III

Dans cette partie, on se propose d'entraîner un PointNet. Pour ce faire, on utilisera une fonction de coût spécifique (voir cellule ci-dessous).

**Consignes :**

1) Entraîner un PointNet sur quelques centaines d'époques.

2) Afficher à chaque époque la justesse des prédictions

3) Charger les poids d'un réseau entraîné sur 500 époques, stockés dans le fichier **pointnet_500_ep.pth** du répertoire https://huggingface.co/nanopiero/pointnet_igloos.

Visualiser les sorties de ce modèle-là en complétant le la dernière cellule de code du calepin.


In [ ]:
optimizer = torch.optim.Adam(pointnet.parameters(),
                             lr=0.0001, betas=(0.9, 0.999))

# manually set alpha weights
alpha = np.array([0.2, 0.8])
gamma = 1
loss_fn = PointNetSegLoss(alpha=alpha, gamma=gamma, dice=True).cuda()

# exemple d'utilisation de PointNetSegLoss:
# La transposition permet de repasser la dimension relative
# aux probabilités en dernier, comme avec torch.nn.CrossEntropyLoss
proba_pred_list = outputs.transpose(1,2)
loss_fn(proba_pred_list, target_list)

In [ ]:
batch_size = 64
n_epochs = 200
n_batch_per_epoch = 10


for epoch in range(1, n_epochs):
  print('epoch : ', epoch)
  for batch in range(1,n_batch_per_epoch):
    ...

In [ ]:
input_points, target_list , target_points = gen_pointcloud(6)

# Il faut construire les prédictions.
proba_pred_list, _, _ = pointnet2.cuda()(input_points.to(device))
pred_list = proba_pred_list.transpose(1,2).max(1)[1].cpu()

# Accuracy:
...
# Tracé

for i in range(6):
  print(i)
  plot_triplets(input_points[i].transpose(0,1), elev=75, azim=0)
  plot_triplets(target_points[i].transpose(0,1),
                title='Cibles',
                cbar_label='classe')
  plot_triplets(...,
                title='Predictions',
                cbar_label='classe')
